# Proyecto 7: Deploy de sistema de recomendación con Watson

En este proyecto llevaremos a cabo la puesta en producción del modelo entrenado en el proyecto 5. Es decir, lo subirmos la nube de IBM y utilizando llamados a la API de Watson tendremos acceso a él para realizar predicciones.

In [1]:
import warnings
warnings.filterwarnings("ignore")
import sklearn
from sklearn.datasets import load_files
moviedir = r'./dataset/movie_reviews' 
movie_reviews = load_files(moviedir, shuffle=True)

In [2]:
#La línea de código no se ejecuta por estar repetida luego. 
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(
#     movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

In [3]:
from sklearn.externals import joblib
eclf = joblib.load('sentiment.pkl') 

C:\Users\Juan\Anaconda3\envs\watson\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


IBM Watson

**1) Cargá** la biblioteca `WatsonMachineLearningAPIClient`

In [4]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**2) Creá** variable con las credenciales que necesita `Watson`. Ellas son: `url, access_key, username, password e instance_id`

In [5]:
#Tuve problemas para crear la variable del modo que lo plantea la platafroma de Acámica y aquí en la entrega. Leyendo la documentación de WMLC llegué a esta manera de autenticarme pero se que existen otras válidas. 
wml_credentials = {
                   "url": "https://us-south.ml.cloud.ibm.com",
                   "apikey":"9tQdDSy6xNcv1BGyWVj0DjMRnNS4DPBh-PerLC_G95yt",
                   "instance_id": "2072fb41-7299-41e2-8aca-f18270b29dee"
                  }

**3) Declará** la variable `client` y guardá en ella al objeto `WatsonMachineLearningAPIClient` con las credenciales como parámetro

In [6]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [7]:
client

**4) Creá** una variable que guarde las propiedades del modelo. Datos del autor y nombre del proyecto.

In [8]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "Juan Francisco Falcone", 
               client.repository.ModelMetaNames.NAME: "Publicación de Modelos"}

**5) Hacé** un pipeline que contenga como primer paso a un `TfidfVectorizer` y como segundo paso, al mejor modelo que hayas obtenido en el proyecto 5. **Entrená** con este pipeline.

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

In [10]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
vector = TfidfVectorizer()
pipeline = make_pipeline(vector, eclf)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_i...00, random_state=None))],
         flatten_transform=None, n_jobs=-1, voting='hard', weights=None))])

**6) Subí** al modelo a IBM Cloud usando `client.repository.store_model` con los parámetros correctos.

In [11]:
published_model = client.repository.store_model(model=pipeline, 
                                                meta_props=model_props, 
                                                training_data=X_train, 
                                                training_target=y_train)

**7) Obtené** el `uid` del modelo y guardalo en una variable.

In [12]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)

In [13]:
models_details = client.repository.list_models()

------------------------------------  ----------------------  ------------------------  -----------------
GUID                                  NAME                    CREATED                   FRAMEWORK
0d256693-1324-4656-b84f-b951d09eeb21  Publicación de Modelos  2020-02-27T21:35:18.525Z  scikit-learn-0.19
9b89bd9b-0e3f-4dbb-a9bd-f7189604a6b8  Publicación de Modelos  2020-02-27T20:52:41.685Z  scikit-learn-0.19
9683489e-5bff-48b6-99b6-4c5c6a3d2519  Reviews classification  2020-02-27T19:34:47.973Z  scikit-learn-0.19
------------------------------------  ----------------------  ------------------------  -----------------


**8) Cargá** el modelo basado en su `uid` y utilizalo para realizar la predicción sobre el conjunto de test

In [14]:
loaded_model = client.repository.load(published_model_uid)
test_predictions = loaded_model.predict(X_test)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished
C:\Users\Juan\Anaconda3\envs\watson\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


**9) Mostrar** el `classification_report` obtenido

In [15]:
from sklearn.metrics import classification_report

print(classification_report(y_test, test_predictions))

             precision    recall  f1-score   support

          0       0.81      0.92      0.86       200
          1       0.91      0.79      0.84       200

avg / total       0.86      0.85      0.85       400

